In [1]:
import cv2
import numpy as np

In [2]:
def show(img, offset=1):
    shape = img.shape
#     offset = 1
    s = 0
    if shape[0] > 1000:
        s = int(10/(int(str(shape[0])[0])+offset))*0.1
    elif shape[1] > 1000:
        s = int(10/(int(str(shape[1])[0])+offset))*0.1
    if s:
        img = cv2.resize(img, None, fx=s, fy=s)
    cv2.imshow("Img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
def biggestContour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 5000:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest,max_area

def drawRectangle(img,biggest,thickness):
    cv2.line(img, (biggest[0][0][0], biggest[0][0][1]), (biggest[1][0][0], biggest[1][0][1]), (0, 0, 255), thickness)
    cv2.line(img, (biggest[0][0][0], biggest[0][0][1]), (biggest[2][0][0], biggest[2][0][1]), (0, 0, 255), thickness)
    cv2.line(img, (biggest[3][0][0], biggest[3][0][1]), (biggest[2][0][0], biggest[2][0][1]), (0, 0, 255), thickness)
    cv2.line(img, (biggest[3][0][0], biggest[3][0][1]), (biggest[1][0][0], biggest[1][0][1]), (0, 0, 255), thickness)
 
    return img

def reorder(myPoints):
 
    myPoints = myPoints.reshape((4, 2))
    myPointsNew = np.zeros((4, 1, 2), dtype=np.int32)
    add = myPoints.sum(1)
 
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] =myPoints[np.argmax(add)]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] =myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
 
    return myPointsNew

In [12]:
base = cv2.imread('2.jpg')
cover = cv2.imread('c5.jpg')

In [5]:
cover.shape

(2560, 1872, 3)

In [6]:
cover = cv2.rotate(cover, cv2.ROTATE_90_COUNTERCLOCKWISE)

In [7]:
show(base)

In [13]:
img = cover.copy()

In [14]:
imgGray = cv2.cvtColor(cover, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1) # ADD GAUSSIAN BLUR

imgThreshold = cv2.Canny(imgBlur,50,500) # APPLY CANNY BLUR
kernel = np.ones((5, 5))
imgDial = cv2.dilate(imgThreshold, kernel, iterations=2) # APPLY DILATION
imgThreshold = cv2.erode(imgDial, kernel, iterations=1)  # APPLY EROSION

## FIND ALL COUNTOURS
imgContours = img.copy() # COPY IMAGE FOR DISPLAY PURPOSES
imgBigContour = img.copy() # COPY IMAGE FOR DISPLAY PURPOSES
contours, hierarchy = cv2.findContours(imgThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # FIND ALL CONTOURS
cv2.drawContours(imgContours, contours, -1, (0, 0, 255), 10) # DRAW ALL DETECTED CONTOURS
 
    
# FIND THE BIGGEST COUNTOUR
biggest, maxArea = biggestContour(contours) # FIND THE BIGGEST CONTOUR

heightImg = 640
widthImg  = 480
print(biggest.size)
if biggest.size != 0:
    biggest=reorder(biggest)
    cv2.drawContours(imgBigContour, biggest, -1, (0, 0, 255), 20) # DRAW THE BIGGEST CONTOUR
    imgBigContour = drawRectangle(imgBigContour,biggest,2)
    pts1 = np.float32(biggest) # PREPARE POINTS FOR WARP
    pts2 = np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]]) # PREPARE POINTS FOR WARP
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))

0


In [15]:
# show(cover)
show(imgBigContour)

In [16]:
resized_cover = cv2.resize(cover, (984, 1372))
show(resized_cover)

In [17]:
large_img = base.copy()
small_img = resized_cover.copy()

In [18]:
n = np.ones_like(small_img)
n=n*255
n[0]

array([[255, 255, 255],
       [255, 255, 255],
       [255, 255, 255],
       ...,
       [255, 255, 255],
       [255, 255, 255],
       [255, 255, 255]], dtype=uint8)

In [19]:
dt = cv2.addWeighted( n, 0.1, small_img, 1, 0)
show(dt)

In [20]:
def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        
        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
    
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

In [21]:
mask = np.ones_like(base)

In [22]:
k = apply_brightness_contrast(small_img, 70, 0)
# show(k)

In [23]:
show(insert_img)

NameError: name 'insert_img' is not defined

In [24]:
insert_img = k.copy()

x_offset=1007
y_offset=304

x_end = x_offset+small_img.shape[1]
y_end = y_offset+small_img.shape[0]

In [25]:
mask[y_offset:y_end, x_offset:x_end] = insert_img 

large_img[y_offset:y_end, x_offset:x_end] = insert_img 
show(large_img)

In [260]:
show(mask)

In [26]:
alpha=0.4
dt = cv2.addWeighted( base, alpha, mask, 1-alpha, 0) 
show(dt)

In [27]:
r = cv2.addWeighted( n, 0.1, small_img, 1, 0)

In [28]:
show(r)